In [2]:
import btrack
from btrack.utils import import_HDF, import_JSON

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import numpy as np

import pandas as pd

from btrack.dataio import import_all_tracks_JSON, HDF5FileHandler
from btrack.constants import Fates
import os


323

# import from JSON

In [ ]:
path = '/home/nathan/data/fucci/fucci1_171201/tracks/tracks_cellpose'

tracks = import_all_tracks_JSON(path, cell_type = 'None')

In [14]:
#print(tracks)
print(tracks[6].fate)
#print(tracks[0].x)
#filter_tracks = [track for track in tracks if track.fate != Fates.TERMINATE_BORDER]
filter_tracks = [track for track in tracks if len(track) > 1000]

track_lengths = [len(track) for track in filter_tracks] 
print(track_lengths)
print(filter_tracks)
print(filter_tracks[0].x)

Fates.TERMINATE_BACK
[1091, 1092, 1062, 1030]
[<btrack.btypes.Tracklet object at 0x7f296fed1450>, <btrack.btypes.Tracklet object at 0x7f296f40bcd0>, <btrack.btypes.Tracklet object at 0x7f296f42b490>, <btrack.btypes.Tracklet object at 0x7f296f43b110>]
[55.0, 49.0, 45.0, 38.0, 33.0, 28.232065636096316, 23.46413127219263, 18.696196908288947, 30.0, 36.0, 40.44423495762109, 43.0, 45.0, 44.0, 45.0, 44.0, 52.0, 55.0, 54.0, 51.0, 45.0, 40.0, 37.0, 35.0, 37.0, 34.0, 33.0, 34.0, 31.0, 36.0, 36.0, 39.0, 43.0, 46.0, 48.0, 49.0, 55.0, 59.0, 63.0, 67.0, 71.0, 75.0, 77.0, 80.0, 85.0, 88.0, 88.0, 92.0, 93.0, 98.0, 104.0, 105.0, 111.0, 112.0, 114.0, 115.0, 114.0, 110.0, 108.0, 103.0, 102.0, 104.0, 107.0, 109.0, 109.0, 112.0, 113.0, 113.0, 116.0, 116.0, 116.0, 115.0, 119.0, 122.0, 120.0, 117.0, 118.0, 116.0, 113.0, 111.0, 109.0, 111.0, 113.0, 115.0, 119.0, 118.0, 119.0, 116.0, 120.0, 123.0, 124.0, 123.0, 125.0, 129.0, 131.0, 130.0, 133.0, 130.0, 129.0, 123.0, 116.0, 116.0, 115.0, 113.0, 113.0, 111.0, 10

### histogram of track lengths

In [ ]:
#size, scale = 1000, 10
tracks_df = pd.Series(track_lengths)
print(max(tracks_df))

tracks_df.plot.hist(grid=True, bins=25, rwidth=0.9,
                   color='#607c8e')
title = 'Distribution of track lengths cellpose'
plt.title(title)
plt.xlabel('Track length')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.75)
fn = os.path.join(path, title)
plt.savefig(fn)

# import from HDF5

In [16]:
filename ='/home/nathan/data/fucci/fucci1_171201/tracks/tracks_cellpose/cellpose_nuclear.h5'
with btrack.dataio.HDF5FileHandler(filename, 'r') as h:
    tracks = h.tracks

[INFO][2020/07/21 01:51:15 PM] Opening HDF file: /home/nathan/data/fucci/fucci1_171201/tracks/tracks_cellpose/cellpose_nuclear.h5
[INFO][2020/07/21 01:51:15 PM] Loading tracks: obj_type_1...
[INFO][2020/07/21 01:51:15 PM] Loading obj_type_1 (160396, 5) (160396 filtered: None)...
[INFO][2020/07/21 01:51:17 PM] Closing HDF file: /home/nathan/data/fucci/fucci1_171201/tracks/tracks_cellpose/cellpose_nuclear.h5


In [1]:
objects = import_HDF('/home/nathan/data/fucci/fucci1_171201/tracks/tracks_cellpose/cellpose_nuclear.h5', filter_using=None)
# needs more attn

NameError: name 'import_HDF' is not defined

In [19]:
print(len(tracks[0]))

2033


In [ ]:
print(objects[0])
print(type(objects))
btrack.BayesianTracker()

## get segmentation from corrupted h5 file

In [ ]:
from skimage import io 
from arboretum.io import ArboretumHDFHandler
with ArboretumHDFHandler('/home/nathan/data/fucci/HDF/tracks_v5_state_labels.1.h5') as h:
    segmentation = h.segmentation
io.imsave('seg.tif', segmentation)

## Get the tracks

In [3]:
# NOTE(arl):  This should be from your image segmentation code
objects = import_HDF('/home/nathan/data/fucci/fucci1_171201/tracks/tracks_cellpose/cellpose_nuclear.h5', filter_using=None)


print(objects)
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

  # configure the tracker using a config file
  tracker.configure_from_file('/home/nathan/analysis/BayesianTracker/models/cell_config.json')

  # append the objects to be tracked
  tracker.append(objects)

  # set the volume (Z axis volume is set very large for 2D data)
  tracker.volume=((0,1024),(0,1024),(-1e5,1e5))

  # track them (in interactive mode)
  tracker.track_interactive(step_size=100)

  # generate hypotheses and run the global optimiser
  tracker.optimize()

  # get the tracks as a python list
  tracks = tracker.tracks

[INFO][2020/07/21 12:46:11 PM] Opening HDF file: /home/nathan/data/fucci/fucci1_171201/tracks/tracks_cellpose/cellpose_nuclear.h5
[INFO][2020/07/21 12:46:11 PM] Loading obj_type_1 (160396, 5) (160396 filtered: None)...
[INFO][2020/07/21 12:46:13 PM] Closing HDF file: /home/nathan/data/fucci/fucci1_171201/tracks/tracks_cellpose/cellpose_nuclear.h5
[INFO][2020/07/21 12:46:13 PM] btrack (v0.3.9) library imported
[INFO][2020/07/21 12:46:13 PM] Starting BayesianTracker session
[INFO][2020/07/21 12:46:13 PM] Loading configuration file: /home/nathan/analysis/BayesianTracker/models/cell_config.json
[INFO][2020/07/21 12:46:13 PM] Loading motion model: b'cell_motion'
[INFO][2020/07/21 12:46:13 PM] Set volume to ((0, 1024), (0, 1024), (-100000.0, 100000.0))
[INFO][2020/07/21 12:46:13 PM] Starting tracking... 
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.i

In [4]:
# get the first track
track_zero = tracks[0]

# print the length of the track
print(len(track_zero))

# print all of the xyzt positions in the track
print(track_zero.x)
print(track_zero.y)
print(track_zero.z)
print(track_zero.t)

# print the fate of the track
print(track_zero.fate)

# print the track ID, root node, parent node and children
print(track_zero.ID)
print(track_zero.root)
print(track_zero.parent)
print(track_zero.children)

133
[115.0, 115.0, 115.0, 105.0, 101.0, 98.0, 95.0, 93.0, 92.0, 97.0, 99.0, 109.0, 112.0, 115.0, 116.0, 113.0, 111.0, 112.0, 110.0, 106.0, 104.0, 100.0, 93.0, 91.0, 91.0, 87.0, 86.0, 87.0, 85.0, 87.0, 88.0, 90.0, 92.0, 94.0, 97.0, 100.0, 104.0, 109.0, 111.0, 115.0, 119.0, 124.0, 126.0, 132.0, 135.0, 139.0, 140.0, 141.0, 144.0, 148.0, 152.0, 150.0, 154.0, 153.0, 152.0, 152.0, 150.0, 146.0, 143.0, 139.0, 141.0, 142.0, 145.0, 147.0, 146.0, 150.0, 153.0, 152.0, 153.0, 153.0, 152.0, 151.0, 150.0, 150.0, 154.0, 153.0, 158.0, 156.0, 152.0, 150.0, 148.0, 150.0, 151.0, 155.0, 158.0, 157.0, 158.0, 154.0, 157.0, 160.0, 164.0, 166.0, 168.0, 171.0, 174.0, 172.0, 172.0, 169.0, 165.0, 157.0, 148.0, 143.0, 140.0, 136.0, 135.0, 131.0, 131.0, 124.0, 121.0, 119.0, 119.0, 117.0, 121.0, 116.0, 120.0, 121.0, 122.0, 126.0, 127.0, 137.0, 142.0, 148.0, 152.0, 156.0, 154.0, 158.0, 160.0, 162.0, 163.0, 165.0, 167.0, 168.0, 161.0]
[303.0, 303.0, 303.0, 292.0, 294.0, 300.0, 302.0, 305.0, 304.0, 305.0, 306.0, 299.0

### visualize the tracks

In [5]:
### length of tracks 
for i in range(len(tracks)):
    print("track no.",i,"length=", len(tracks[i]))
    

track no. 0 length= 133
track no. 1 length= 69
track no. 2 length= 22
track no. 3 length= 5
track no. 4 length= 107
track no. 5 length= 64
track no. 6 length= 1091
track no. 7 length= 2
track no. 8 length= 386
track no. 9 length= 55
track no. 10 length= 349
track no. 11 length= 23
track no. 12 length= 75
track no. 13 length= 178
track no. 14 length= 629
track no. 15 length= 52
track no. 16 length= 57
track no. 17 length= 96
track no. 18 length= 21
track no. 19 length= 7
track no. 20 length= 94
track no. 21 length= 865
track no. 22 length= 86
track no. 23 length= 25
track no. 24 length= 602
track no. 25 length= 37
track no. 26 length= 160
track no. 27 length= 12
track no. 28 length= 1
track no. 29 length= 558
track no. 30 length= 23
track no. 31 length= 128
track no. 32 length= 12
track no. 33 length= 111
track no. 34 length= 25
track no. 35 length= 58
track no. 36 length= 609
track no. 37 length= 49
track no. 38 length= 59
track no. 39 length= 591
track no. 40 length= 22
track no. 41 l

track no. 1360 length= 1
track no. 1361 length= 349
track no. 1362 length= 2
track no. 1363 length= 348
track no. 1364 length= 1
track no. 1365 length= 9
track no. 1366 length= 1
track no. 1367 length= 10
track no. 1368 length= 4
track no. 1369 length= 346
track no. 1370 length= 41
track no. 1371 length= 1
track no. 1372 length= 1
track no. 1373 length= 346
track no. 1374 length= 1
track no. 1375 length= 3
track no. 1376 length= 3
track no. 1377 length= 345
track no. 1378 length= 345
track no. 1379 length= 345
track no. 1380 length= 1
track no. 1381 length= 4
track no. 1382 length= 1
track no. 1383 length= 1
track no. 1384 length= 1
track no. 1385 length= 1
track no. 1386 length= 1
track no. 1387 length= 1
track no. 1388 length= 117
track no. 1389 length= 1
track no. 1390 length= 1
track no. 1391 length= 1
track no. 1392 length= 2
track no. 1393 length= 339
track no. 1394 length= 1
track no. 1395 length= 3
track no. 1396 length= 6
track no. 1397 length= 338
track no. 1398 length= 2
tra

In [ ]:
mpl.rcParams['legend.fontsize'] = 10


fig = plt.figure()
ax = fig.gca(projection='3d')

track_34 = tracks[51]
z = list(range(len(track_34)))

x = track_34.x
y = track_34.y
ax.plot(x, y, z, label='track_34')
ax.legend()

plt.show()

# output .hdf5

In [ ]:
fn = '/home/nathan/analysis/BayesianTracker/examples/segmented_v5.hdf5'
tracker.export(fn,obj_type='obj_type_1')

# do all of the above

In [ ]:
import btrack
from btrack.utils import import_HDF
from btrack.utils import import_JSON

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import numpy as np

# NOTE(arl):  This should be from your image segmentation code
objects = import_HDF('/home/nathan/data/fucci/HDF/segmented.hdf5')

print(objects)
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

    # configure the tracker using a config file
    tracker.configure_from_file('/home/nathan/analysis/BayesianTracker/models/cell_config_1.json')

    # append the objects to be tracked
    tracker.append(objects)

    # set the volume (Z axis volume is set very large for 2D data)
    tracker.volume=((0,1024),(0,1024),(-1e5,1e5))

    # track them (in interactive mode)
    tracker.track_interactive(step_size=100)

    # generate hypotheses and run the global optimiser
    tracker.optimize()

    # get the tracks as a python list
    tracks = tracker.tracks

    fn = '/home/nathan/data/fucci/HDF/segmented.hdf5'
    tracker.export(fn,obj_type='obj_type_1')
    
    print("tick")


In [3]:
import arboretum

from skimage import io
#seg = io.imread('/home/nathan/data/fucci/other_mask_data/segmented_v5.tif')
arboretum.run()

/home/nathan/anaconda3/lib/python3.7/site-packages/napari/utils/dask_utils.py:173: UserWarning: For best performance with Dask arrays in napari, please upgrade Dask to v2.15.0 or later. Current version is 2.11.0
  'For best performance with Dask arrays in napari, please '
[INFO][2020/08/04 12:20:36 pm] Loading configuration file: /home/nathan/analysis/tracker/cell_config.json


AssertionError: 

KeyboardInterrupt: 

# export to xml 

In [ ]:
def export_to_FIJI(filename, tracks, trees=[]):
    import json
    import xml.etree.cElementTree as ET
    root = ET.Element("data", name=filename)
    for trk in tracks:
        trajectory = ET.SubElement(root, "trajectory", id=str(int(trk.ID)))
        ET.SubElement(trajectory, "root").text = str(trk.root)
        ET.SubElement(trajectory, "length").text = str(len(trk))
        ET.SubElement(trajectory, "fate").text = str(trk.fate.value)
        ET.SubElement(trajectory, "x").text = str([float("{0:2.1f}".format(x)) for x in trk.x])
        ET.SubElement(trajectory, "y").text = str([float("{0:2.1f}".format(y)) for y in trk.y])
        ET.SubElement(trajectory, "n").text = str([int(t) for t in trk.t])
        ET.SubElement(trajectory, "class").text = str([l.lower() for l in trk.label])
        if trk.parent != 0:
            # ET.SubElement(trajectory, "parent").text = str(trk.parent-1)
            # NOTE(arl): need to check whether this -1 on the index is required
            ET.SubElement(trajectory, "parent").text = str(trk.parent)
        else:
            ET.SubElement(trajectory, "parent").text = str(None)
        if trk.children:
            ET.SubElement(trajectory, "children").text = str(trk.children)
    for tree in trees:
        lin_tree = lineage.linearise_tree(tree)
        if len(lin_tree) > 1:
            root_node = lin_tree.pop(0)
            this_tree = ET.SubElement(root, "tree", root_id=str(root_node.ID))
            for child in lin_tree:
                ET.SubElement(this_tree, "child").text = str(child.ID)
    XMLtree = ET.ElementTree(root)
    XMLtree.write(filename)

In [ ]:
export_to_FIJI('/home/nathan/analysis/segmented.xml', tracks)